From explore.ipynb we can find the images that we can augment by rotating, flipping or mirroring.
## TODO

Ideas:

- Balance data by augmentation (Create more new pictures for underrepresented signs)
- Assign different signs to different augmentation methods --> What is possible for which method?

Signs:
Fill in all possible signs

Methods to augment data + for which signs applicable:

-Flipping (horizontal/vertical)
    --> check possible invariances (e.g turn left signs) 
        and signs which can then be detected as other signs

-Rotating

## TODO

In [1]:
fippable_horizontally = [1, 5]